# Hyperparameter Tuning

## Experiment
The goal of the third experiment is to find the best combination of hyperparameters across a range of different regressor models. The experiment assumes the following:

- A time window of 1 year is chosen, starting with 1. April, 2017, thus covering all seasons. April of 2017 was the first whole month with Level-2A imagery and the closest to the recording dates.
- Level-2A Sentinel 2 satellite imagery and indices are used as input features, using the reducers and numbers of composites determined by the previous experiment.
- The evaluation metric for the models is RMSE (Root Mean Squared Error).

The hyperparameter optimization is done using Bayesian Optimization with Optuna. The optuna studies and the best pipeline obects are saved using dill instead of pickle to support lambda functions.

At first we train a baseline model using simple data. We use a random forest model with default parameters and use all spectral Level-2A bands (starting with "B") averaged across the study timewindow as the data:

In [ ]:
# Create a baseline composite from a Sentinel image with the average across 1 year
from ltm.data import sentinel_composite
from datetime import datetime
from pathlib import Path
from sklearn.metrics import make_scorer, root_mean_squared_error

data_path = "../data/processed/hyperparameter_tuning/data_mean_1.tif"
target_path = "../data/processed/target.tif"

# Create the composite if it does not exist
if not Path(data_path).exists():
    Path(data_path).parent.mkdir(parents=True, exist_ok=True)
    sentinel_composite(
        target_path_from=target_path,
        data_path_to=data_path,
        time_window=(datetime(2017, 4, 1), datetime(2018, 4, 1)),
    )

In [ ]:
# Cross validate a default random forest regressor
from sklearn.ensemble import RandomForestRegressor
from ltm.features import load_raster, drop_nan_rows
from sklearn.model_selection import cross_validate

scorer = make_scorer(
    root_mean_squared_error
)

data = load_raster(data_path)
target = load_raster(target_path)
data, target = drop_nan_rows(data, target)

cv_result = cross_validate(
    RandomForestRegressor(n_jobs=-1, random_state=42),
    data,
    target,
    cv=5,
    scoring=scorer,
    n_jobs=-1,
)
baseline_score = cv_result["test_score"].mean()

print(f"Baseline RMSE: {baseline_score}")

We use optuna with `n_jobs=1` for reproducibility. The number of trials is set to 100 for all decision tree ensembles and 500 for the other models.

In [ ]:
# Define the metric and load the data
from sklearn.metrics import make_scorer, root_mean_squared_error
from ltm.features import load_raster, drop_nan_rows

# Define the metric
rmse_scorer = make_scorer(
    root_mean_squared_error, greater_is_better=False
)

# Load the data
data_path = "../data/processed/data.tif"
target_path = "../data/processed/target.tif"
data = load_raster(data_path)
target = load_raster(target_path)

# Drop rows with NaN in target
data, target = drop_nan_rows(data, target)

In [ ]:
# Create save folder and wrapper functions
from pathlib import Path

save_folder = "../models/"
Path(save_folder).mkdir(parents=True, exist_ok=True)


def suggest_categorical(*args, **kwargs):
    return "suggest_categorical", args, kwargs


def suggest_discrete_uniform(*args, **kwargs):
    return "suggest_discrete_uniform", args, kwargs


def suggest_float(*args, **kwargs):
    return "suggest_float", args, kwargs


def suggest_int(*args, **kwargs):
    return "suggest_int", args, kwargs

In [ ]:
from skelm import ELMRegressor
from ltm.models import hyperparam_search

elm_default = ELMRegressor(random_state=42)
search_space = [
    suggest_float("alpha", 1e-8, 1e5, log=True),
    suggest_categorical("include_original_features", [True, False]),
    suggest_float("n_neurons", 1, 1000),
    suggest_categorical("ufunc", ["tanh", "sigm", "relu", "lin"]),
    suggest_float("density", 0.01, 0.99),
]

elm_model, elm_study = hyperparam_search(
    elm_default,
    search_space,
    data,
    target,
    rmse_scorer,
    n_trials=500,
    save_folder=save_folder,
    random_state=42,
)
elm_model

In [ ]:
# K-Nearest Neighbour
from sklearn.neighbors import KNeighborsRegressor
from ltm.models import hyperparam_search

knn_default = KNeighborsRegressor(n_jobs=-1)
search_space = [
    suggest_int("n_neighbors", 1, 100),
    suggest_categorical("weights", ["uniform", "distance"]),
    suggest_categorical("algorithm", ["auto", "ball_tree", "kd_tree", "brute"]),
]

knn_model, knn_study = hyperparam_search(
    knn_default,
    search_space,
    data,
    target,
    rmse_scorer,
    n_trials=500,
    save_folder=save_folder,
    random_state=42,
)
knn_model

In [ ]:
# SGD Linear Regression
from sklearn.linear_model import SGDRegressor
from ltm.models import hyperparam_search

sgd_default = SGDRegressor(random_state=42)
search_space = [
    suggest_categorical(
        "loss",
        [
            "squared_error",
            "huber",
            "epsilon_insensitive",
            "squared_epsilon_insensitive",
        ],
    ),
    suggest_float("alpha", 1e-6, 1e5, log=True),
    suggest_float("l1_ratio", 0, 1),
]

sgd_model, sgd_study = hyperparam_search(
    sgd_default,
    search_space,
    data,
    target,
    rmse_scorer,
    n_trials=500,
    save_folder=save_folder,
    random_state=42,
)
sgd_model

In [ ]:
# Support Vector Machine
from sklearn.svm import SVR
from ltm.models import hyperparam_search

svr_default = SVR()
search_space = [
    suggest_float("C", 1e-5, 1e5, log=True),
    suggest_float("epsilon", 1e-5, 1e5, log=True),
    suggest_categorical("kernel", ["poly", "rbf", "sigmoid"]),
]

svr_model, svr_study = hyperparam_search(
    svr_default,
    search_space,
    data,
    target,
    rmse_scorer,
    n_trials=500,
    save_folder=save_folder,
    random_state=42,
)
svr_model

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from ltm.models import hyperparam_search

et_default = ExtraTreesRegressor(n_jobs=-1, random_state=42)
search_space = [
    suggest_int("n_estimators", 1, 200),
    suggest_float("min_impurity_decrease", 1e-5, 0.5, log=True),
    suggest_categorical("criterion", ["squared_error", "absolute_error"]),
]

et_model, et_study = hyperparam_search(
    et_default,
    search_space,
    data,
    target,
    rmse_scorer,
    n_trials=100,
    save_folder=save_folder,
    random_state=42,
)
et_model

In [ ]:
# HistGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from ltm.models import hyperparam_search

hgbr_default = HistGradientBoostingRegressor(random_state=42)
search_space = [
    suggest_int("max_iter", 100, 1000),
    suggest_float("learning_rate", 0.001, 0.5, log=True),
    suggest_int("max_leaf_nodes", 2, 1000),
    suggest_categorical("l2_regularization", [0, 1e-10, 1e-5, 1e-3, 1e-1, 1]),
]

hgbr_model, hgbr_study = hyperparam_search(
    hgbr_default,
    search_space,
    data,
    target,
    rmse_scorer,
    n_trials=100,
    save_folder=save_folder,
    random_state=42,
)
hgbr_model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from ltm.models import hyperparam_search

rf_default = RandomForestRegressor(n_jobs=-1, random_state=42)
search_space = [
    suggest_int("n_estimators", 1, 200),
    suggest_int("max_depth", 1, 1000),
    suggest_float("max_features", 0.1, 1.0),
    suggest_float("min_samples_split", 1e-5, 0.5, log=True),
    suggest_float("min_samples_leaf", 1e-5, 0.5, log=True),
    suggest_categorical("bootstrap", [True, False]),
    suggest_categorical(
        "criterion",
        ["squared_error", "absolute_error", "poisson", "friedman_mse"],
    ),
]

rf_model, rf_study = hyperparam_search(
    rf_default,
    search_space,
    data,
    target,
    rmse_scorer,
    n_trials=100,
    save_folder=save_folder,
    random_state=42,
)
rf_model

In [ ]:
from xgboost import XGBRegressor
from ltm.models import hyperparam_search

xgb_default = XGBRegressor(n_jobs=-1, random_state=42)
search_space = [
    suggest_int("n_estimators", 10, 200),
    suggest_int("max_depth", 1, 20),
    suggest_float("learning_rate", 0.001, 0.5, log=True),
    suggest_float("gamma", 0, 0.5),
    suggest_int("min_child_weight", 1, 11),
]

xgb_model, xgb_study = hyperparam_search(
    xgb_default,
    search_space,
    data,
    target,
    rmse_scorer,
    n_trials=100,
    save_folder=save_folder,
    random_state=42,
)
xgb_model

In [ ]:
import matplotlib.pyplot as plt
import scienceplots

plt.style.use("science")

In [ ]:
# Plot the hyperparam search scores during search
studies = [
    elm_study,
    knn_study,
    sgd_study,
    svr_study,
    et_study,
    hgbr_study,
    rf_study,
    xgb_study,
]

fig, axs = plt.subplots(2, 4, figsize=(20, 10))

for ax, study in zip(axs.flat, studies):
    trials = study.trials_dataframe()
    ax.plot(-trials["value"], label="Score")
    ax.set_title(study.study_name)
    ax.set_xlabel("Trial")
    ax.set_ylabel("RMSE Score")
    ax.axhline(-study.best_value, color="g", linestyle="--", label="Best score")
    ax.set_ylim(0, 1)
    ax.legend()

In [ ]:
import pandas as pd
from collections import defaultdict
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from tqdm.notebook import tqdm

csv_path = "../reports/hyperparameter_tuning.csv"
scoring = {
    "Mean Absolute Error": make_scorer(mean_absolute_error),
    "Root Mean Squared Error": make_scorer(root_mean_squared_error),
    "R2 Score": make_scorer(r2_score),
}

tuned_models = [
    elm_model,
    knn_model,
    sgd_model,
    svr_model,
    rf_model,
    et_model,
    hgbr_model,
    xgb_model,
]

if not Path(csv_path).exists():
    # Create columns
    model_names = [model.steps[-1][1].__class__.__name__ for model in tuned_models]
    columns = defaultdict(list)
    columns["Model"] = model_names

    # Cross validate default and tuned models
    for model in tqdm(tuned_models):
        cv_result = cross_validate(
            model, data, target, scoring=scoring, n_jobs=-1
        )

        for metric in scoring.keys():
            columns[metric].append(cv_result[f"test_{metric}"].mean())

    # Create dataframe
    df = pd.DataFrame(columns)
    df.set_index("Model", inplace=True)
    df.to_csv(csv_path)
else:
    df = pd.read_csv(csv_path, index_col="Model")

# Sort the dataframe by RMSE
df.sort_values("Root Mean Squared Error")

The best model is the XGBRegressor with a RMSE of 0.251. We visualize the quality of the predictions by predicting the study area using cross validation and plotting the predictions against the actual values.

In [ ]:
# Plot ground truth
import rasterio
import matplotlib as mpl

# Prepare the colormap
cmap = mpl.cm.viridis
norm = mpl.colors.Normalize(vmin=0, vmax=1)
mappable = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)

# Create subplot and colorbar
fig, ax = plt.subplots()
fig.colorbar(mappable, ax=ax, label="Conifer Proportion", shrink=0.8)

# Load the target raster
with rasterio.open(target_path) as src:
    target_raster = src.read()

# Plot the target raster
ax.imshow(target_raster.transpose(1, 2, 0), interpolation="nearest", cmap=cmap, norm=norm)
ax.axis("off")
ax.set_title("Ground Truth")

plt.show()

In [ ]:
from ltm.models import cv_predict

fig, axs = plt.subplots(2, 4, figsize=(12, 3))
fig.colorbar(mappable, ax=axs, label="Conifer Proportion", shrink=0.8)

prediction_rasters = []
for ax, model in zip(axs.flat, tuned_models):
    prediction_raster = cv_predict(model, data_path, target_path)
    prediction_rasters.append(prediction_raster)
    
    im = ax.imshow(prediction_raster.transpose(1, 2, 0), interpolation="nearest", cmap=cmap, norm=norm)
    ax.axis("off")
    ax.set_title(model.steps[-1][1].__class__.__name__)

plt.show()

In [ ]:
# Plot prediction raster of XGBoost to target raster
plt.plot(target_raster.flatten(), prediction_rasters[-1].flatten(), ".", alpha=0.05)
plt.gca().set_aspect("equal")
plt.plot([0, 1], [0, 1], "--", color="k", alpha=0.5)

plt.title("XGBoost")
plt.xlabel("Ground Truth")
plt.ylabel("Prediction")

plt.show()